In [12]:
import cv2
import numpy as np

# Load template and defect images
template_image = cv2.imread('template.jpg', cv2.IMREAD_GRAYSCALE)
defect_image = cv2.imread('image.jpg', cv2.IMREAD_GRAYSCALE)

# Convert images to floating point representation
template_image_float = template_image.astype(np.float32)
defect_image_float = defect_image.astype(np.float32)

# Perform image registration using phase correlation
shift = cv2.phaseCorrelate(template_image_float, defect_image_float)

# Apply translation shift to defect image
rows, cols = defect_image.shape
M = np.float32([[1, 0, shift[0][0]], [0, 1, shift[0][1]]])
defect_image_aligned = cv2.warpAffine(defect_image, M, (cols, rows))

# Compute absolute difference between aligned defect image and template image
difference_image = cv2.absdiff(template_image, defect_image_aligned)

# Apply threshold to highlight significant differences
threshold = 50  # Adjust threshold as needed
difference_image[difference_image < threshold] = 0
difference_image[difference_image >= threshold] = 255

# Visualize the difference image
cv2.imshow('Difference Image', difference_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
# Load template and defect images
template_image = cv2.imread(r"E:\4th sem\Maths\project\pcb\DeepPCB-master\PCBData\group00041\00041\00041000_temp.jpg", cv2.IMREAD_GRAYSCALE)
defect_image = cv2.imread(r"E:\4th sem\Maths\project\pcb\DeepPCB-master\PCBData\group00041\00041\00041000_test.jpg", cv2.IMREAD_GRAYSCALE)


# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = sift.detectAndCompute(template_image, None)
keypoints2, descriptors2 = sift.detectAndCompute(defect_image, None)

# Initialize matcher
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# Apply ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Extract matched keypoints
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Estimate perspective transformation
M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 6)

# Apply perspective transformation to align defect image
rows, cols = defect_image.shape
defect_image_aligned = cv2.warpPerspective(defect_image, M, (cols, rows))

# Compute absolute difference between aligned defect image and template image
difference_image = cv2.absdiff(template_image, defect_image_aligned)

# Apply threshold to highlight significant differences
threshold = 175  # Adjust threshold as needed
difference_image[difference_image < threshold] = 0
difference_image[difference_image >= threshold] = 255

# Visualize the difference image
cv2.imshow('Difference Image', difference_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
# Load template and defect images
template_image = cv2.imread(r"E:\4th sem\Maths\project\pcb\DeepPCB-master\PCBData\group00041\00041\00041000_temp.jpg", cv2.IMREAD_GRAYSCALE)
defect_image = cv2.imread(r"E:\4th sem\Maths\project\pcb\DeepPCB-master\PCBData\group00041\00041\00041000_test.jpg", cv2.IMREAD_GRAYSCALE)


# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = sift.detectAndCompute(template_image, None)
keypoints2, descriptors2 = sift.detectAndCompute(defect_image, None)

# Initialize matcher
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# Apply ratio test
good_matches = []
for m, n in matches:
    if m.distance < .95 * n.distance:
        good_matches.append(m)

# Extract matched keypoints
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Estimate perspective transformation using RANSAC
M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 6.0)

# Apply perspective transformation to align defect image
rows, cols = defect_image.shape
defect_image_aligned = cv2.warpPerspective(defect_image, M, (cols, rows))

# Convert keypoints to NumPy array of floats
keypoints2_array = np.array([kp.pt for kp in keypoints2], dtype=np.float32)

# Refine alignment using Lucas-Kanade optical flow
nextPts, status, _ = cv2.calcOpticalFlowPyrLK(defect_image_aligned, template_image, keypoints2_array, None)

# Filter keypoints with valid flow
good_new = nextPts[status[:,0] == 1]
good_old = keypoints2_array[status[:,0] == 1]

# Compute absolute difference between refined aligned defect image and template image
difference_image = cv2.absdiff(template_image, defect_image_aligned)

# Apply threshold to highlight significant differences
threshold = 50  # Adjust threshold as needed
difference_image[difference_image < threshold] = 0
difference_image[difference_image >= threshold] = 255

# Visualize the difference image
cv2.imshow('Difference Image', difference_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite("final.jpg",difference_image.astype(np.uint8))


True

In [3]:
import cv2
import numpy as np

# Read the image
image = cv2.imread('final.jpg')

# Convert the image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define lower and upper bounds for white color
lower_white = np.array([0, 0, 200])
upper_white = np.array([180, 255, 255])

# Threshold the image to get only white colors
mask = cv2.inRange(hsv, lower_white, upper_white)

# Replace white pixels with red
image[mask == 255] = [0, 0, 255]

# Show the result
cv2.imshow('Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the result
# cv2.imwrite('result_image.jpg', image)


In [4]:
import cv2
import numpy as np

# Load template and defect images
template_image = cv2.imread('final.jpg')
defect_image = cv2.imread('edges100.jpg')

# Add the two images
result_image = cv2.add(template_image, defect_image)

# Display the result
cv2.imshow('Result Image', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
import cv2
import numpy as np

# Load template and defect images
template_image = cv2.imread('final.jpg', cv2.IMREAD_GRAYSCALE)
defect_image = cv2.imread('edges100.jpg', cv2.IMREAD_GRAYSCALE)

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = sift.detectAndCompute(template_image, None)
keypoints2, descriptors2 = sift.detectAndCompute(defect_image, None)

# Initialize matcher
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# Apply ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Ensure we have enough good matches for homography estimation
if len(good_matches) < 4:
    print("Not enough good matches for homography estimation.")
    exit()

# Extract matched keypoints
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Estimate perspective transformation using RANSAC
M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

# Apply perspective transformation to align defect image
rows, cols = defect_image.shape
defect_image_aligned = cv2.warpPerspective(defect_image, M, (cols, rows))

# Add template image and defect image
result_image = cv2.add(template_image, defect_image_aligned)

# Visualize the result image
cv2.imshow('Result Image', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Not enough good matches for homography estimation.


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'cv::findHomography'


: 

In [58]:
import cv2
import numpy as np

# Load template and defect images
template_image = cv2.imread('template.jpg', cv2.IMREAD_GRAYSCALE)
defect_image = cv2.imread('image.jpg', cv2.IMREAD_GRAYSCALE)

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
keypoints1, descriptors1 = sift.detectAndCompute(template_image, None)
keypoints2, descriptors2 = sift.detectAndCompute(defect_image, None)

# Initialize matcher
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# Apply ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Extract matched keypoints
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Estimate perspective transformation using RANSAC
M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

# Apply perspective transformation to align defect image
rows, cols = defect_image.shape
defect_image_aligned = cv2.warpPerspective(defect_image, M, (cols, rows))

# Convert keypoints to NumPy array of floats
keypoints2_array = np.array([kp.pt for kp in keypoints2], dtype=np.float32)

# Refine alignment using Lucas-Kanade optical flow
nextPts, status, _ = cv2.calcOpticalFlowPyrLK(defect_image_aligned, template_image, keypoints2_array, None)

# Filter keypoints with valid flow
good_new = nextPts[status[:,0] == 1]
good_old = keypoints2_array[status[:,0] == 1]

# Compute absolute difference between refined aligned defect image and template image
difference_image = cv2.absdiff(template_image, defect_image_aligned)

# Create a red mask with the same shape as the difference image
red_mask = np.zeros_like(difference_image, dtype=np.uint8)
red_mask[:] = (0, 0, 255)  # Set intensity to red in RGB color space

# Apply the red mask to the difference image
red_difference_image = cv2.addWeighted(difference_image, 1, red_mask, 1, 0)

# Visualize the red difference image
cv2.imshow('Red Difference Image', red_difference_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the red difference image
cv2.imwrite("red_difference.jpg", red_difference_image)


ValueError: could not broadcast input array from shape (3,) into shape (640,640)